In [1]:
from utils.models import GMF, MLP, NeuFM
from utils.dataset import RatingsDataset
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm

In [2]:
users = 3974
movies = 3564

dataloader = DataLoader(
    RatingsDataset(
        "/home/nubol23/Desktop/Codes/USP/SCC5966/kaggle/data/train_data.csv",
        "user_id",
        "movie_id",
        "rating",
    ), 
    batch_size=1024,
    num_workers=12
)

In [3]:
import torch
import torch.nn as nn
import torch.optim as opt

In [4]:
# model = GMF(users, movies, 5).cuda()
# model = MLP(users, movies, 150, [100, 50, 20]).cuda()

# model = NeuFM(
#     GMF(users, movies, 5),
#     MLP(users, movies, 50, [20, 10]),
#     alpha=0.5,
# ).cuda()

model = NeuFM(
    GMF(users, movies, 8),
    MLP(users, movies, 16, [16, 8]),
    alpha=0.5,
).cuda()

model

NeuFM(
  (h): Linear(in_features=16, out_features=1, bias=False)
  (gmf): GMF(
    (P): Embedding(3974, 8)
    (Q): Embedding(3564, 8)
    (h): Identity()
  )
  (mlp): MLP(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): Linear(in_features=16, out_features=8, bias=True)
    )
    (h): Identity()
  )
)

In [5]:
criterion = nn.MSELoss().cuda()
# optimizer = opt.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
optimizer = opt.Adam(model.parameters(), lr=0.005)

for epoch in tqdm(range(15)):
    n_batches = len(dataloader)
    avg_loss = 0
    for i_batch, (vus, vis, rs) in enumerate(dataloader):
        vus = vus.cuda()
        vis = vis.cuda()
        rs = rs.cuda()
        
        optimizer.zero_grad()
        y_hat = model(vus, vis)
        
        loss = criterion(y_hat, rs)
        loss.backward()
        
        optimizer.step()
        
        avg_loss += float(loss.detach().cpu())
    print(avg_loss/n_batches)

  0%|          | 0/15 [00:00<?, ?it/s]

1.9855452188439953
0.8885492501026802
0.8298564409936657
0.8122035477680104
0.8689546857512634
0.8540474959570943
0.8136694832840039
0.8077013753638923
0.7586306944944476
0.6939768823048541
0.6804616081578131
0.6883360151457423
0.6868850897059186
0.7275477177427925
0.7001948936749961


In [6]:
with torch.no_grad():
    k = 0
    for vus, vis, rs in dataloader:
        if k == 10:
            break
        
        vus = vus.cuda()
        vis = vis.cuda()
        rs = rs.cuda()
        
        pred = torch.clip(model(vus, vis), 1, 5)
        
        print(pred.ravel())
        print(rs.ravel())
        print()
        
        k += 1

tensor([4.5011, 3.6050, 4.2378,  ..., 4.5140, 3.8894, 2.5216], device='cuda:0')
tensor([5., 3., 4.,  ..., 4., 5., 3.], device='cuda:0')

tensor([3.6454, 2.1259, 1.7618,  ..., 2.8171, 3.4809, 4.2134], device='cuda:0')
tensor([3., 2., 3.,  ..., 2., 4., 3.], device='cuda:0')

tensor([3.8363, 3.1497, 2.6374,  ..., 2.1573, 1.2289, 2.2628], device='cuda:0')
tensor([4., 2., 1.,  ..., 5., 1., 3.], device='cuda:0')

tensor([1.0000, 1.4061, 1.8877,  ..., 3.3614, 3.5249, 4.3694], device='cuda:0')
tensor([1., 1., 1.,  ..., 3., 3., 4.], device='cuda:0')

tensor([4.8997, 4.9617, 3.4223,  ..., 2.2679, 1.5330, 1.4333], device='cuda:0')
tensor([4., 5., 3.,  ..., 2., 1., 1.], device='cuda:0')

tensor([2.2482, 1.0288, 3.1948,  ..., 3.5372, 3.5879, 5.0000], device='cuda:0')
tensor([1., 1., 4.,  ..., 2., 4., 5.], device='cuda:0')

tensor([3.7750, 4.7802, 3.4227,  ..., 4.2099, 3.6523, 4.7603], device='cuda:0')
tensor([5., 5., 3.,  ..., 4., 4., 5.], device='cuda:0')

tensor([4.6031, 4.0741, 5.0000,  ..., 2.1

## Test

In [7]:
test_dataloader = DataLoader(
    RatingsDataset(
        "/home/nubol23/Desktop/Codes/USP/SCC5966/kaggle/data/test_data.csv",
        "user_id",
        "movie_id",
    ), 
    batch_size=1024,
    num_workers=12
)

In [8]:
test_predictions = []

with torch.no_grad():
    k = 0
    for vus, vis in test_dataloader:
#         if k == 10:
#             break
        
        vus = vus.cuda()
        vis = vis.cuda()
        
        pred = torch.clip(model(vus, vis), 1, 5).cpu().ravel().tolist()
        test_predictions += pred
        
        k += 1

In [9]:
import pandas as pd

test_csv = pd.read_csv("../../data/test_data.csv")

In [10]:
out_df = pd.DataFrame.from_dict(
    {
        "id": list(test_csv["id"]),
        "rating": test_predictions
    }
)

out_df.head()

,id,rating
0,0,3.177693
1,1,2.616074
2,2,2.464490
3,3,3.379605
4,4,1.119932


In [11]:
out_df.to_csv(f"outputs_csv/neumf_test_2.csv", index=False)